In [4]:
import bq_helper
from bq_helper import BigQueryHelper
stackoverflow = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                         dataset_name="stackoverflow")

hackernews = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                         dataset_name="hacker-news")

Using Kaggle's public dataset BigQuery integration.
Using Kaggle's public dataset BigQuery integration.


In [5]:
bq_assistant = BigQueryHelper("bigquery-public-data", "stackoverflow")
bq_assistant.list_tables()

Using Kaggle's public dataset BigQuery integration.


['badges',
 'comments',
 'post_history',
 'post_links',
 'posts_answers',
 'posts_moderator_nomination',
 'posts_orphaned_tag_wiki',
 'posts_privilege_wiki',
 'posts_questions',
 'posts_tag_wiki',
 'posts_tag_wiki_excerpt',
 'posts_wiki_placeholder',
 'stackoverflow_posts',
 'tags',
 'users',
 'votes']

In [6]:
from google.cloud import bigquery
client = bigquery.Client()

# use the client object to run a query
query = """
    SELECT *
    FROM `bigquery-public-data.stackoverflow.posts_questions`
    LIMIT 10
"""
results = client.query(query).to_dataframe()

# print the results
print(results)

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


         id                                              title  \
0  64177830  How to read to a certain point in python for e...   
1  64183547                            How to use nested "map"   
2  64220954          Azure DevOps Ephemeral agents not working   
3  64228509        How do load value from DB only once in java   
4  64232071  gh-pages script cannot commit .nojekyll to GitHub   
5  64237197  AttributeError: type object 'Graph' has no att...   
6  64244371  IAR Compiler Linker Error [Li009]: _CPP_Except...   
7  64256016       Nativescript gestures not working with vuejs   
8  64262142  Instantiate and set to parent of a sub gameObject   
9  64263183               SLIM PUT-Request CORS Header missing   

                                                body  accepted_answer_id  \
0  <p>I tried to read from every line until the <...                 NaN   
1  <p>I would like to map a json, based on some n...          64183941.0   
2  <p>I am trying to configure an Ephemeral a

2. How many posts were created in 2021?

In [7]:
query2 = """SELECT COUNT(*) as num_posts_2021
FROM `bigquery-public-data.stackoverflow.posts_questions`
WHERE EXTRACT(YEAR FROM creation_date) = 2021"""

response2 = stackoverflow.query_to_pandas_safe(query2, max_gb_scanned=10)
response2.head(10)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,num_posts_2021
0,1629580


3. What is the average score of all the posts in the dataset?

In [8]:
query3 = """SELECT AVG(score) as avg_score
FROM `bigquery-public-data.stackoverflow.posts_questions`"""

response3 = stackoverflow.query_to_pandas_safe(query3, max_gb_scanned=10)
response3.head(10)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,avg_score
0,2.214833


4. How many posts have a score greater than or equal to 500?

In [9]:
query4 = """SELECT COUNT(*) as num_posts_500plus
FROM `bigquery-public-data.stackoverflow.posts_questions`
WHERE score >= 500"""

response4 = stackoverflow.query_to_pandas_safe(query4)
response4.head(10)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,num_posts_500plus
0,5106


5. What is the average reputation of users who ask questions on weekdays vs. weekends?

In [10]:
query5 = """WITH question_reputations AS (
  SELECT q.owner_user_id, 
         AVG(u.reputation) AS user_reputation, 
         EXTRACT(DAYOFWEEK FROM q.creation_date) AS day_of_week
  FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
  JOIN `bigquery-public-data.stackoverflow.users` AS u
    ON q.owner_user_id = u.id
  WHERE q.owner_user_id IS NOT NULL
  GROUP BY q.owner_user_id, day_of_week
)
SELECT day_of_week,
       AVG(CASE WHEN day_of_week IN (2, 3, 4, 5, 6) THEN user_reputation END) AS weekday_avg_reputation,
       AVG(CASE WHEN day_of_week IN (1, 7) THEN user_reputation END) AS weekend_avg_reputation
FROM question_reputations
GROUP BY day_of_week"""

response5 = stackoverflow.query_to_pandas_safe(query5)
response5.head(10)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,day_of_week,weekday_avg_reputation,weekend_avg_reputation
0,4,762.435557,NaN
1,2,795.486569,NaN
2,3,770.324633,NaN
3,7,NaN,966.263111
4,6,812.957719,NaN
5,1,NaN,962.818933
6,5,769.261242,NaN


6. What is the reputation and badge count of users across different tenures on StackOverflow?

In [11]:
query6 = """SELECT User_Tenure,
       COUNT(1) AS Num_Users,
       ROUND(AVG(reputation)) AS Avg_Reputation,
       ROUND(AVG(num_badges)) AS Avg_Num_Badges
FROM (
  SELECT users.id AS user,
         ROUND(TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), ANY_VALUE(users.creation_date), DAY)/365) AS user_tenure,
         ANY_VALUE(users.reputation) AS reputation,
         SUM(IF(badges.user_id IS NULL, 0, 1)) AS num_badges
  FROM `bigquery-public-data.stackoverflow.users` users
  LEFT JOIN `bigquery-public-data.stackoverflow.badges` badges
  ON users.id = badges.user_id
  GROUP BY user
)
GROUP BY User_Tenure
ORDER BY User_Tenure;
        """
response6 = stackoverflow.query_to_pandas_safe(query6, max_gb_scanned=10)
response6.head(10)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,User_Tenure,Num_Users,Avg_Reputation,Avg_Num_Badges
0,0.0,258913,2.0,0.0
1,1.0,3042923,3.0,1.0
2,2.0,2490716,5.0,1.0
3,3.0,2073247,10.0,1.0
4,4.0,1656413,17.0,1.0
5,5.0,1688614,24.0,1.0
6,6.0,1657249,35.0,2.0
7,7.0,1412133,57.0,3.0
8,8.0,1210540,83.0,3.0
9,9.0,1167849,119.0,4.0


7.What are 10 of the “easier” gold badges to earn?

In [12]:
query7 = """SELECT badge_name AS First_Gold_Badge,
       COUNT(1) AS Num_Users,
       ROUND(AVG(tenure_in_days)) AS Avg_Num_Days
FROM
(
  SELECT
    badges.user_id AS user_id,
    badges.name AS badge_name,
    TIMESTAMP_DIFF(badges.date, users.creation_date, DAY) AS tenure_in_days,
    ROW_NUMBER() OVER (PARTITION BY badges.user_id
                       ORDER BY badges.date) AS row_number
  FROM
    `bigquery-public-data.stackoverflow.badges` badges
  JOIN
    `bigquery-public-data.stackoverflow.users` users
  ON badges.user_id = users.id
  WHERE badges.class = 1
)
WHERE row_number = 1
GROUP BY First_Gold_Badge
ORDER BY Num_Users DESC
LIMIT 10;
        """

response7 = stackoverflow.query_to_pandas_safe(query7, max_gb_scanned=10)
response7.head(10)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,First_Gold_Badge,Num_Users,Avg_Num_Days
0,Famous Question,418859,1635.0
1,Fanatic,27621,863.0
2,Great Answer,25005,1969.0
3,Unsung Hero,21079,825.0
4,Electorate,12123,1242.0
5,Populist,10930,1716.0
6,Publicist,2748,2409.0
7,Steward,1565,1200.0
8,Great Question,710,876.0
9,Copy Editor,344,751.0


## Hackernews database

1. Find the average number of comments per post, grouped by month and year, for all posts with at least 100 points.

In [13]:
qnews = """WITH post_data AS (
  SELECT 
    id,
    EXTRACT(YEAR FROM timestamp) AS year,
    EXTRACT(MONTH FROM timestamp) AS month,
    score,
    IFNULL(descendants, 0) AS comments
  FROM `bigquery-public-data.hacker_news.full`
  WHERE type = 'story' AND score >= 100
)
SELECT 
  year,
  month,
  AVG(comments) AS avg_comments
FROM post_data
GROUP BY year, month
ORDER BY year, month
        """

answer1 = hackernews.query_to_pandas_safe(qnews, max_gb_scanned=10)
answer1.head(10)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,year,month,avg_comments
0,2007,2,824.000000
1,2007,4,71.000000
2,2007,6,42.000000
3,2007,7,92.000000
4,2007,8,116.666667
5,2007,9,79.666667
6,2007,10,102.500000
7,2007,11,66.000000
8,2007,12,50.000000
9,2008,1,60.500000
